PURPOSE OF CODE

Train data empty phrases are cleaned and same phrase rows are merged as their sentiment scores are averaged.

PROCESS NOTES

WILL BE DONE

DONE

- empty rows are assigned as NaN and erased
- same phrases with different sentiment scores are avaraged and merged as single

In [30]:
import numpy as np
import pandas as pd
df=pd.read_csv('data/train.csv',sep="\t")
df = df.drop('Unnamed: 0', 1)
df.head()

,Unnamed: 0.1,PhraseId,SentenceId,Phrase,Sentiment
0,0,20685,923,encourag see threedimension averag middleag wo...,3
1,1,134482,7254,uncomfort time relev,3
2,2,71670,3657,film drumbeat authent,2
3,3,116520,6213,sit enjoy certain level forget,2
4,4,106992,5649,wellmean patron mask social injustic least rep...,1


Replace empty phrase rows to NaN then delete.

In [31]:
#print("size of data with empty phrase rows=",len(df.index))

#define a map to change empty phrases to NaN
def change_nan(x):
    if isinstance(x,str):
        words=x.split()
        if len(words)>0:
            return x
    return np.nan

df['Phrase'] = df.Phrase.map(change_nan)

#remove NaN rows of df
df=df.dropna()

#reset indexing
df=df.reset_index(drop=True)

#print("size of data after empty phrase rows deleted=",len(df.index))

In [32]:
df.head()

,Unnamed: 0.1,PhraseId,SentenceId,Phrase,Sentiment
0,0,20685,923,encourag see threedimension averag middleag wo...,3
1,1,134482,7254,uncomfort time relev,3
2,2,71670,3657,film drumbeat authent,2
3,3,116520,6213,sit enjoy certain level forget,2
4,4,106992,5649,wellmean patron mask social injustic least rep...,1


Afer clean, the same same phrases with differen sentiment scores created as follows.

In [33]:
same=[]
for x in df.Phrase:
    same=df.Phrase[df.Phrase==x]
    if len(same.index)>1:
        break
df.loc[same.index[0]]


Unnamed: 0.1                                                    0
PhraseId                                                    20685
SentenceId                                                    923
Phrase          encourag see threedimension averag middleag wo...
Sentiment                                                       3
Name: 0, dtype: object

In [34]:
df.loc[same.index[1]]

Unnamed: 0.1                                                15857
PhraseId                                                    20686
SentenceId                                                    923
Phrase          encourag see threedimension averag middleag wo...
Sentiment                                                       4
Name: 46823, dtype: object

Take a phrase as referance to the "phrase" list. Find the same ones in data and record the indexes to the "same" list. In such same phrase groups assign the first one as referance by changing its sentiment score to group average and change other phrases to NaN to remove later from the list. Before doing that sentiment scores should be converted into float data type from intiger(as a default).

In [35]:
#print("data type of sentiment scores= ",df.Sentiment.dtype)

#convert sentiment scores to float to write averaged ones
df.Sentiment=df.Sentiment.astype(float)
df.dtypes
#print("data type of sentiment scores after conversion= ",df.Sentiment.dtype)

#write size of data before merge
#print("data size before merge= ",len(df.index))

#"phrase" holds the current phrase to compare through whole set and find the indexes of same other phrase rows 
#which will written in "same"
#then they will changed to NaN for further clean process
#summation holds the summation of same phrase sentiment scores that will avaraged and written to the first phrase among all same phrases
same=[]
summation=0
phrase=[]

for index,row in df.iterrows():
    if row.Phrase!=np.nan :
        same=df.Phrase[df.Phrase==row.Phrase]
        if len(same.index)>1:
            summation=0
            phrase=df.loc[same.index[0],'Phrase']
            for x in range(1,len(same.index)):
                df.loc[same.index[x],'Phrase']=np.nan
                summation+=df.loc[same.index[x],'Sentiment']
            summation+=df.loc[same.index[0],'Sentiment']
            df.loc[same.index[0],'Sentiment']=summation/len(same.index)

#clean all NaN rows
df=df.dropna()

#write size of data after merge
#print("data size after merge= ",len(df.index))

Remove NaN assigneds and reindex data.

In [36]:
df=df.dropna()
df=df.reset_index(drop=True)
df.head(15)

,Unnamed: 0.1,PhraseId,SentenceId,Phrase,Sentiment
0,0,20685,923,encourag see threedimension averag middleag wo...,3.666667
1,1,134482,7254,uncomfort time relev,3.000000
2,2,71670,3657,film drumbeat authent,2.000000
3,3,116520,6213,sit enjoy certain level forget,2.000000
4,4,106992,5649,wellmean patron mask social injustic least rep...,1.666667
5,5,145547,7912,mani,2.000000
6,6,103413,5452,profund,2.500000
7,7,140286,7613,persuad,2.666667
8,8,105348,5562,compel plot,3.000000
9,9,90212,4695,darkest varieti,1.250000


This cleaned data is sort of vocabulary list. Save file as train-clean.csv under data folder.

In [37]:
df.to_csv("data/train-clean.csv", sep='\t')
df.describe()

,Unnamed: 0.1,PhraseId,SentenceId,Sentiment
count,75879.000000,75879.000000,75879.000000,75879.000000
mean,14945.514253,79189.822902,4142.949419,2.060726
std,9010.900459,44780.366035,2490.551511,0.880129
min,0.000000,2.000000,1.000000,0.000000
25%,7046.000000,40683.500000,1950.000000,1.666667
50%,14626.000000,79569.000000,4096.000000,2.000000
75%,22657.000000,118101.000000,6308.000000,2.666667
max,31211.000000,156060.000000,8544.000000,4.000000


In [38]:
print("done succesfully")

done succesfully
